# Бейзлайн для задачи goodsification (VTB Data Fusion 2021)

In [81]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [82]:
#conda uninstall -c anaconda py-xgboost
#conda install -c bioconda xgboost=0.6a2
#conda install -c conda-forge xgboost 

In [83]:
#pip install fastparquet
#conda install -c conda-forge python-snappy

In [84]:
train = pd.read_parquet('data_fusion_train.parquet')

## Убираем ненужные теги

In [85]:
train = train[train.category_id != -1] #неразмеченные товары
train = train.drop_duplicates('item_name')

In [86]:
train = train.drop(['receipt_id','receipt_dayofweek','receipt_time','item_quantity'], axis=1) #в данной тетради будем использовать только текстовое поле 'item_name'

## Усиливаем item_name с помощью первого слова, чтобы повысить общий вес наиболее "достоверного" слова в строке

In [87]:
train['first_word'] = train['item_name'].apply(lambda x: x.split()[0] if len(x.split()) > 0 else 'none')

In [88]:
train['item_name'] = train['item_name'] + ' ' + train['first_word']

In [89]:
train['item_name']

1                                       Молоко 3,2%,шт Молоко
3                            Компот из изюма, 114 ккал Компот
4           Макаронные изделия отварные (масло сливочное),...
17                         Кофе Капучино Большой Эден 18 Кофе
40                                Хлеб на СЫВОРОТКЕ 350г Хлеб
                                  ...                        
45669181               Напиток Энерг. Ред Булл 0,355л Напиток
45681543                          Хеменгуэй Дайкири Хеменгуэй
45690702    Пиво светлое "Халзан" 4,5 % об, пл/б. 1,5 л(шт...
45692298                            Экспресс педикюр Экспресс
45700308         Конфеты Харитоша 1кг мол. ваф Яшкино Конфеты
Name: item_name, Length: 48225, dtype: object

## Убираем цифры и дубликаты пробелов

In [90]:
import re
train['item_name'] = train['item_name'].str.replace('\d+', '') #цифры
train['item_name'] = train['item_name'].str.lower() 
train['item_name'] = train['item_name'].apply(lambda x: re.sub(r'[^\w]', ' ', str(x))) #символы
train['item_name'] = train['item_name'].apply(lambda x: re.sub(' +', ' ', str(x))) #дубликаты пробелов

In [91]:
#токенизация для дальнейшей обработки

import nltk
nltk.download('punkt')
from nltk import word_tokenize
train['item_name_token'] = train['item_name'].apply(lambda x: word_tokenize(str(x)))

[nltk_data] Downloading package punkt to /Users/ilya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Выделяем корни во всех словах, чтобы "молоко" и "молок" не воспринимались разными словами (стемминг)

In [92]:
from nltk.stem import SnowballStemmer

snowball = SnowballStemmer(language='russian')
train['item_name_token'] = train['item_name_token'].apply(lambda x: [snowball.stem(y) for y in x])

## Английские наименования также убираем, чтобы они не путали классификатор

In [93]:
#pip install alphabet-detector

In [94]:
from alphabet_detector import AlphabetDetector
ad = AlphabetDetector()

def transf(input):
    output =[]
    for i in range(len(input)):
        if ad.only_alphabet_chars(input[i],'CYRILLIC') == True:
            output.append(input[i])
    return output


train['item_name_token'] = train['item_name_token'].apply(lambda x: transf(x))

In [95]:
train['item_name_token']

1                                          [молок, шт, молок]
3                            [компот, из, изюм, ккал, компот]
4           [макарон, издел, отварн, масл, сливочн, ккал, ...
17                             [коф, капучин, больш, эд, коф]
40                              [хлеб, на, сыворотк, г, хлеб]
                                  ...                        
45669181              [напиток, энерг, ред, булл, л, напиток]
45681543                         [хеменгуэ, дайкир, хеменгуэ]
45690702           [пив, светл, халза, об, пл, б, л, шт, пив]
45692298                        [экспресс, педикюр, экспресс]
45700308       [конфет, харитош, кг, мол, ваф, яшкин, конфет]
Name: item_name_token, Length: 48225, dtype: object

In [96]:
train['item_name_token'] = train['item_name_token'].apply(lambda x: ' '.join(x)) #сливаем все обратно в одну строку для векторизации

In [97]:
train

,item_name,item_price,item_nds_rate,category_id,brands,first_word,item_name_token
1,молоко шт молоко,8,2,78,None,Молоко,молок шт молок
3,компот из изюма ккал компот,4,1,71,None,Компот,компот из изюм ккал компот
4,макаронные изделия отварные масло сливочное кк...,4,1,71,None,Макаронные,макарон издел отварн масл сливочн ккал макарон
17,кофе капучино большой эден кофе,12,1,70,None,Кофе,коф капучин больш эд коф
40,хлеб на сыворотке г хлеб,7,-1,84,None,Хлеб,хлеб на сыворотк г хлеб
...,...,...,...,...,...,...,...
45669181,напиток энерг ред булл л напиток,10,6,83,None,Напиток,напиток энерг ред булл л напиток
45681543,хеменгуэй дайкири хеменгуэй,15,6,0,None,Хеменгуэй,хеменгуэ дайкир хеменгуэ
45690702,пиво светлое халзан об пл б л шт пиво,10,6,0,None,Пиво,пив светл халза об пл б л шт пив
45692298,экспресс педикюр экспресс,15,6,42,None,Экспресс,экспресс педикюр экспресс


## В список stopwords добавляем "шумные" токены

In [98]:
from nltk.corpus import stopwords 

nltk.download('words')
#wordsEng = set(nltk.corpus.words.words())

nltk.download('stopwords')
stop = stopwords.words('russian')
stop = stop.extend(['к','в','ип','ооо','ш','д','а','б','в','е',
            'ё','ж','з','и','й','к','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ю','я','a','b','c','d','e','f',
           'd','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','y',
            'z','new','без','ок','бкк','классическ','на','упак','бел','черн','кдв','зпп','мф','из','особ','по','нов','рецептур','премиум','мк','оа','ая','эдишн','ип'
            ])

[nltk_data] Downloading package words to /Users/ilya/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ilya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.75) # здесь ограничиваем веса тех токенов, которые встречаются в словаре слишком часто

X_sparse = tfidf.fit_transform(train['item_name_token'])
X_nsparse = train.drop('item_name', axis=1)

In [105]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sparse, train.category_id.values, test_size=0.33, random_state=1)

In [106]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

clf = LinearSVC(max_iter=2000, C=2.1)
clf.fit(X_train, y_train)

print(cross_val_score(clf, X_sparse, train.category_id.values, cv=3))

[0.8237014  0.8007465  0.76609642]


## Итоговый скор бейзлайна с использованием только item_name получается ~80%